<a href="https://colab.research.google.com/github/audreyemmely/pln/blob/main/pln_lista2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Resolução Lista 2**

---

Audrey Emmely Rodrigues Vasconcelos

Karen Nayara Gomes da Silva